In [1]:
import pandas as pd
import numpy as np
import scipy.io
import math
from collections import Counter
import copy
import warnings
from tensorflow.keras.layers import Dense, Input
from sklearn.preprocessing import StandardScaler
from tensorflow.keras import Model
from tensorflow.keras.optimizers import Adam
warnings.filterwarnings("ignore")

In [2]:
def most_frequent(List):
    occurence_count = Counter(List)
    return occurence_count.most_common(1)[0][0]
def rms(arr):
    n = len(arr)
    squared = np.array(arr) * np.array(arr)
    sum = np.sum(squared)
    mean = (sum / (float)(n))
    root = math.sqrt(mean)
    return root
def mav(arr):
    n = len(arr)
    mav = sum((abs(np.array(arr))))/(float)(n)
    return mav
def var(arr):
    n = len(arr)
    squared = np.array(arr) * np.array(arr)
    sum = np.sum(squared)
    result = (sum / (float)(n))
    return result
def wl(arr):
    n = len(arr)
    sum = 0
    for i in range(1, n):
        sum += abs(arr[i] - arr[i - 1])
    return sum
def iav(arr):
    return sum(abs(np.array(arr)))
def mean(arr):
    return np.sum(arr)/len(arr)
def pretty(d, indent=0):
    for key, value in d.items():
        print('\t' * indent + str(key))
        if isinstance(value, dict):

            pretty(value, indent + 2)
            print('-----------------------')
        else:
            print('\t' * (indent + 2) + str(value))
            print(" ")
def consecutive(data, stepsize=1):
    return np.split(data, np.where(np.diff(data) != stepsize)[0] + 1)
def extractSubject(name):
    ex1Path = 'DB1/' + name + '/' + name + '_A1_E1.mat'
    print(ex1Path)
    ex1 = scipy.io.loadmat(ex1Path)
    emg = ex1['emg']
    EMGdf = pd.DataFrame.from_dict(emg)
    stimulus = ex1['restimulus']

    ex2Path = 'DB1/' + name + '/' + name + '_A1_E2.mat'
    ex2 = scipy.io.loadmat(ex2Path)
    emg2 = ex2['emg']
    EMGdf2 = pd.DataFrame.from_dict(emg2)
    stimulus2 = ex2['restimulus']

    ex3Path = 'DB1/' + name + '/' + name + '_A1_E3.mat'
    ex3 = scipy.io.loadmat(ex3Path)
    emg3 = ex3['emg']
    EMGdf3 = pd.DataFrame.from_dict(emg3)
    stimulus3 = ex3['restimulus']

    Movements = {}
    for m in range(1, 51):
        if (m < 11):
            movementIndices = np.where(stimulus == m)[0]
            repetitions = consecutive(movementIndices)
            EMG = EMGdf
        elif (m < 28):
            movementIndices = np.where(stimulus2 == (m - 10))[0]
            repetitions = consecutive(movementIndices)
            EMG = EMGdf2
        else:
            movementIndices = np.where(stimulus3 == (m - 27))[0]
            repetitions = consecutive(movementIndices)
            EMG = EMGdf3

        Electrodes = {}
        for e in range(1, 11):
            temp = {}
            for r in range(1, 7):
                startIndex = repetitions[r - 1][0]
                LastIndex = repetitions[r - 1][len(repetitions[r - 1]) - 1]
                df = EMG.iloc[startIndex:LastIndex, e - 1]
                df.reset_index(drop=True, inplace=True)
                narray = df.to_numpy(dtype=None, copy=False)
                temp["R{0}".format(r)] = narray
            Electrodes["Electrode{0}".format(e)] = temp
        Movements["Movement{0}".format(m)] = Electrodes
    return Movements
def extractSubjectAngles(name):
    ex1Path = 'Kinematics/' + name + '/' + name + '_E1_A1.mat'
    print(ex1Path)
    ex1 = scipy.io.loadmat(ex1Path)
    angles1 = ex1['angles']
    anglesDF1 = pd.DataFrame.from_dict(angles1)
    stimulus = ex1['restimulus']

    ex2Path = 'Kinematics/' + name + '/' + name + '_E2_A1.mat'
    ex2 = scipy.io.loadmat(ex2Path)
    angles2 = ex2['angles']
    anglesDF2 = pd.DataFrame.from_dict(angles2)
    stimulus2 = ex2['restimulus']

    ex3Path = 'Kinematics/' + name + '/' + name + '_E3_A1.mat'
    ex3 = scipy.io.loadmat(ex3Path)
    angles3 = ex3['angles']
    anglesDF3 = pd.DataFrame.from_dict(angles3)
    stimulus3 = ex3['restimulus']

    Movements = {}
    for m in range(1, 51):
        if (m < 11):
            movementIndices = np.where(stimulus == m)[0]
            repetitions = consecutive(movementIndices)
            angle = anglesDF1
        elif (m < 28):
            movementIndices = np.where(stimulus2 == (m - 10))[0]
            repetitions = consecutive(movementIndices)
            angle = anglesDF2
        else:
            movementIndices = np.where(stimulus3 == (m - 27))[0]
            repetitions = consecutive(movementIndices)
            angle = anglesDF3

        Angles = {}
        for a in range(1,23):
            if a == 6:
               Angles["Angle6"] = ""
               continue

            temp={}
            for r in range(1, 7):
                startIndex = repetitions[r - 1][0]
                LastIndex = repetitions[r - 1][len(repetitions[r - 1]) - 1]
                df = angle.iloc[startIndex:LastIndex, a-1]
                df.reset_index(drop=True, inplace=True)
                narray = df.to_numpy(dtype=None, copy=False)
                temp["R{0}".format(r)] = narray
            Angles["Angle{0}".format(a)] = temp
        Movements["Movement{0}".format(m)] = Angles

    return Movements


In [142]:
final_df = pd.DataFrame(columns={'RMS1', 'MAV1', 'VAR1', 'WL1', 'IAV1',
                                 'RMS2', 'MAV2', 'VAR2', 'WL2', 'IAV2',
                                 'RMS3', 'MAV3', 'VAR3', 'WL3', 'IAV3',
                                 'RMS4', 'MAV4', 'VAR4', 'WL4', 'IAV4',
                                 'RMS5', 'MAV5', 'VAR5', 'WL5', 'IAV5',
                                 'RMS6', 'MAV6', 'VAR6', 'WL6', 'IAV6',
                                 'RMS7', 'MAV7', 'VAR7', 'WL7', 'IAV7',
                                 'RMS8', 'MAV8', 'VAR8', 'WL8', 'IAV8',
                                 'RMS9', 'MAV9', 'VAR9', 'WL9', 'IAV9',
                                 'RMS10', 'MAV10', 'VAR10', 'WL10', 'IAV10',
                                 'Train'})
final_df_angle = pd.DataFrame(columns={})
df_angle = pd.DataFrame(columns={})
i=0
for s in range(1,8):
    subject = 'S' + str(s)
    df_angles = pd.DataFrame.from_dict(extractSubjectAngles(subject))
    for m in range(1,51):
        M = df_angles['Movement'+str(m)]
        start = copy.deepcopy(i)
        for a in range (1,23):
            if a==6:
                continue

            Angles = M['Angle' +str(a)]
            for r in range(1,7):
                rep = "R" + str(r)
                for x in range(0, len(Angles[rep]), 48):
                    df_angle.at[i, a-1] = mean(Angles[rep][x:x + 50])
                    i += 1
            if a != 22:
                i = copy.deepcopy(start)
    final_df_angle = final_df_angle.append(df_angle, ignore_index=True)            

Kinematics/S1/S1_E1_A1.mat
Kinematics/S2/S2_E1_A1.mat
Kinematics/S3/S3_E1_A1.mat
Kinematics/S4/S4_E1_A1.mat
Kinematics/S5/S5_E1_A1.mat
Kinematics/S6/S6_E1_A1.mat
Kinematics/S7/S7_E1_A1.mat


In [143]:
for s in range(1,8):
    subject = 'S' + str(s)
    dff = pd.DataFrame.from_dict(extractSubject(subject))
    df = pd.DataFrame(columns={'RMS1', 'MAV1', 'VAR1', 'WL1', 'IAV1',
                           'RMS2', 'MAV2', 'VAR2', 'WL2', 'IAV2',
                           'RMS3', 'MAV3', 'VAR3', 'WL3', 'IAV3',
                           'RMS4', 'MAV4', 'VAR4', 'WL4', 'IAV4',
                           'RMS5', 'MAV5', 'VAR5', 'WL5', 'IAV5',
                           'RMS6', 'MAV6', 'VAR6', 'WL6', 'IAV6',
                           'RMS7', 'MAV7', 'VAR7', 'WL7', 'IAV7',
                           'RMS8', 'MAV8', 'VAR8', 'WL8', 'IAV8',
                           'RMS9', 'MAV9', 'VAR9', 'WL9', 'IAV9',
                           'RMS10', 'MAV10', 'VAR10', 'WL10', 'IAV10',
                           'Train'})
    for e in range(1, 11):
        electrode = 'Electrode' + str(e)
        i=0
        for m in range(1,51):
            M = dff['Movement'+str(m)][electrode]
            for r in range(1, 7):
                rep = "R" + str(r)
                if (r in [1, 3, 4, 6]):
                    train = 1
                else:
                    train = 0
                for x in range(0, len(M[rep]), 48):
                    df.at[i, 'RMS' + str(e)] = rms(M[rep][x:x + 50])
                    df.at[i, 'MAV' + str(e)] = mav(M[rep][x:x + 50])
                    df.at[i, 'VAR' + str(e)] = var(M[rep][x:x + 50])
                    df.at[i, 'WL' + str(e)] = wl(M[rep][x:x + 50])
                    df.at[i, 'IAV' + str(e)] = iav(M[rep][x:x + 50])
                    df.at[i, 'Train'] = train
                    i += 1
    final_df = final_df.append(df, ignore_index=True)

DB1/S1/S1_A1_E1.mat
DB1/S2/S2_A1_E1.mat
DB1/S3/S3_A1_E1.mat
DB1/S4/S4_A1_E1.mat
DB1/S5/S5_A1_E1.mat
DB1/S6/S6_A1_E1.mat
DB1/S7/S7_A1_E1.mat


In [116]:
final_df['CMC1_f'] = df_angle[0]
final_df['CMC1_a'] =  df_angle[1]
final_df['MCP1'] =  df_angle[2]
final_df['IP1'] =  df_angle[3]
final_df['MCP2_f'] =  df_angle[4]
final_df['PIP2'] =  df_angle[6]
final_df['MCP3_f'] =  df_angle[7]
final_df['PIP3'] =  df_angle[8]
final_df['MCP4_f'] =  df_angle[9]
final_df['MCP4_a'] =  df_angle[10]
final_df['PIP4'] =  df_angle[11]
final_df['CMC5'] =  df_angle[12]
final_df['MCP5_f'] =  df_angle[13]
final_df['MCP5_a'] =  df_angle[14]
final_df['PIP5'] =  df_angle[15]
final_df['DIP2'] =  df_angle[16]
final_df['DIP3'] =  df_angle[17]
final_df['DIP4'] =  df_angle[18]
final_df['DIP5'] =  df_angle[19]
# final_df['WRIST_F'] =  df_angle[20]
# final_df['WRIST_A'] =  df_angle[21]

features = {'RMS1', 'MAV1', 'VAR1', 'WL1', 'IAV1',
            'RMS2', 'MAV2', 'VAR2', 'WL2', 'IAV2',
            'RMS3', 'MAV3', 'VAR3', 'WL3', 'IAV3',
            'RMS4', 'MAV4', 'VAR4', 'WL4', 'IAV4',
            'RMS5', 'MAV5', 'VAR5', 'WL5', 'IAV5',
            'RMS6', 'MAV6', 'VAR6', 'WL6', 'IAV6',
            'RMS7', 'MAV7', 'VAR7', 'WL7', 'IAV7',
            'RMS8', 'MAV8', 'VAR8', 'WL8', 'IAV8',
            'RMS9', 'MAV9', 'VAR9', 'WL9', 'IAV9',
            'RMS10', 'MAV10', 'VAR10', 'WL10', 'IAV10', }

angles = {'CMC1_f','CMC1_a','MCP1','IP1','MCP2_f',
        'PIP2','MCP3_f','MCP4_f','MCP4_a',
        'PIP4','CMC5','MCP5_f','MCP5_a','PIP5',
        'DIP2','DIP4','DIP5', 'DIP3', 'PIP3'}

print(final_df)

         IAV3      MAV8    IAV9      MAV6      RMS4     WL5      RMS1  \
0      1.0731  0.053168  2.0183    0.0024  0.002475   0.005   0.03838   
1      8.1079  0.084324  2.5488  0.002748  0.027218   0.059  0.049779   
2       2.412  0.062058  1.0291  0.002794   0.01148  0.0246   0.03652   
3      0.1841   0.03213   0.516    0.0024    0.0024   0.015  0.026781   
4      0.2624   0.04868  1.3248    0.0024  0.002475   0.005   0.03086   
...       ...       ...     ...       ...       ...     ...       ...   
6811   5.5737  0.382858  5.7958  1.282564  0.078815  0.3441  0.004888   
6812  15.0222  0.708252   6.411  0.978954  0.163461  0.3006  0.008156   
6813  23.9407  0.853718  8.3082  0.835404  0.288002  0.5932  0.008498   
6814   3.7321   0.20181  1.0048  0.863278  0.091343  0.2687  0.006017   
6815   0.1009  0.241641  0.2629  0.438856  0.010544   0.178  0.004296   

          VAR4     WL9    IAV1  ...      CMC5     MCP5_f     MCP5_a  \
0     0.000006  0.4762  1.7652  ...  1.928711  54.61

In [117]:
X_train = final_df[final_df['Train'] == 1].loc[:, features]
scalar =  StandardScaler()
scalar = scalar.fit(X_train)
X_train = scalar.transform(X_train)
X_test = final_df[final_df['Train'] == 0].loc[:, features]
X_test = scalar.transform(X_test)

y_train = final_df[final_df['Train'] == 1].loc[:, angles]
y_test = final_df[final_df['Train'] == 0].loc[:, angles]
y_train.to_csv('angles_y_train.csv')

In [141]:
input = Input(shape =(50,))
L1 = Dense(50, activation='tanh')(input)
L2 = Dense(50, activation='sigmoid')(L1)
L3 = Dense(50, activation='sigmoid')(L2)
L4 = Dense(50, activation='sigmoid')(L3)
ouput = Dense(19, activation='linear')(L4)
model = Model(input, ouput)
model.compile(optimizer=Adam(learning_rate=0.0045), loss="mean_squared_error", metrics=['mae', 'mse'])
model.fit(X_train, y_train, epochs=2000)
print(model.evaluate(X_test, y_test))
predictions = model.predict(X_train)



Epoch 1/2000
141/141 [==============================] - 0s 618us/step - loss: 1124.7134 - mae: 21.1391 - mse: 1124.7134
Epoch 2/2000
141/141 [==============================] - 0s 697us/step - loss: 989.4990 - mae: 21.1255 - mse: 989.4990
Epoch 3/2000
141/141 [==============================] - 0s 664us/step - loss: 943.8929 - mae: 20.5179 - mse: 943.8929
Epoch 4/2000
141/141 [==============================] - 0s 630us/step - loss: 910.2203 - mae: 20.1506 - mse: 910.2203
Epoch 5/2000
141/141 [==============================] - 0s 692us/step - loss: 895.0649 - mae: 19.9955 - mse: 895.0649
Epoch 6/2000
141/141 [==============================] - 0s 943us/step - loss: 881.5737 - mae: 19.8160 - mse: 881.5737
Epoch 7/2000
141/141 [==============================] - 0s 671us/step - loss: 870.6559 - mae: 19.6390 - mse: 870.6559
Epoch 8/2000
141/141 [==============================] - 0s 637us/step - loss: 851.1708 - mae: 19.3336 - mse: 851.1708
Epoch 9/2000
141/141 [==============================] 

KeyboardInterrupt: 

In [139]:
print(predictions[0])
print(predictions[100])

[ 44.12652      7.498062     4.93161      5.4727244   -1.1564376
 -11.148542     9.452545     1.7715158  -16.410074    12.515305
  48.729477     6.4476886    2.396341    13.670867    19.219746
  28.821085    19.496334     0.79541075 -17.592176  ]
[ 24.784756   30.894444   -7.7032633   2.395214    5.083597  -82.791534
   7.516862   -7.611231  -32.155087   10.485303   17.052961    1.2658887
   1.9150028   3.6408334   9.571558   -4.3259025   5.0065737  -7.561964
 -10.23494  ]


In [ ]:
54.61345891955365,13.418102414241673,40.187775547894965,1.1888788558435552,4.101825888519285,-25.604314583244467,9.815728089925392,0.7238138389000116,-12.842432671465666,10.081088039618288,90.43685610817194,14.175547125216255,1.9287114778764098,15.766037398552712,19.857298752862327,26.402826169380774,11.695412868578613,0.5178820980259151,-18.842589280331165

27.225043452500504,28.295868195239255,-11.454231605843008,-0.391457008278672,4.253836695248956,-65.89292141911842,9.611844583887207,-6.593315193065799,-38.21023473140616,11.827818390737072,25.854688563497184,6.920610113342703,2.1119786139799888,1.0333574325250479,14.897568576810711,10.73810029759642,9.395141182097142,-0.19959425116956603,-12.523755980490446